### 라이브러리 선언

In [35]:
import pandas as pd
import numpy as np

### 데이터 불러오기

In [36]:
dataUrl = "https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/kopo_channel_seasonality_new.csv"

In [37]:
selloutData = pd.read_csv(dataUrl)

### 1. 데이터 타입 통합

In [38]:
## regionid : str, product: str, yearweek: int 
## qty: float

In [39]:
selloutData.REGIONID = selloutData.REGIONID.astype(str)
selloutData.PRODUCT = selloutData.PRODUCT.astype(str)
selloutData.YEARWEEK = selloutData.YEARWEEK.astype(int)
selloutData.QTY = selloutData.QTY.astype(float)

In [40]:
selloutData["YEAR"] = selloutData.YEARWEEK.astype(str).str[:4]
selloutData["WEEK"] = selloutData.YEARWEEK.astype(str).str[4:]

[불량 데이터 처리]
kopo_channel_seasonality_new.csv 자료를 담은 
selloutData 변수에서
QTY컬럼 음수(반품)인 경우 0, 양수인 경우 기존 QTY 값유지하는 로직을 적용하여 QTY_NEW 컬럼을 추가하세요

In [41]:
selloutData["QTY_NEW"] = np.where( selloutData.QTY < 0, 0, selloutData.QTY )

[데이터 통합]
selloutData 자료에서 
YEAR, WEEK 컬럼을 생성하고 WEEK 가 52 이하인
데이터만 조회한 후 refinedSelloutData 변수에 담으세요


In [42]:
exceptWeek = "53"
refinedSelloutData = selloutData.loc[  selloutData.WEEK != exceptWeek    ]

refinedSelloutData 에서 →
지역, 상품, 연주차 컬럼순으로 오름차순 정렬하여
sortedData 변수에 담으세요


In [22]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]

sortedData = refinedSelloutData.sort_values(by=sortKey)

[지역, 상품, 연도 별 집계]
sortedData 에서 지역, 상품, 연도 단위
판매량(QTY_NEW) 의 평균 연산 후
groupData 변수에 담으세요

이후 컬럼명을 QTY_MEAN로 변경하세요


In [62]:
groupKey = ["REGIONID","PRODUCT","YEAR"]
groupData = sortedData.groupby(by=groupKey)[["QTY_NEW"]].agg("mean").reset_index()
groupData = groupData.rename(columns={"QTY_NEW":"QTY_MEAN"})
groupData

,REGIONID,PRODUCT,YEAR,QTY_MEAN
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538
...,...,...,...,...
2377,A77,PRODUCT1,2015,3030.019231
2378,A77,PRODUCT1,2016,3375.326923
2379,A77,PRODUCT12,2014,2035.788462
2380,A77,PRODUCT12,2015,3540.980769
